In [1]:
#1、加载库
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets,transforms


In [2]:
#2、定义超参数
BATCH_SIZE = 32
DEVICE = torch.device("cpu")
EPOCHS = 4 

In [3]:
#3、构建pipeline,对图像做处理
pipeline = transforms.Compose([
    
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,))
])


In [4]:
#4、下载、加载数据
from torch.utils.data import DataLoader

train_set = datasets.MNIST("data",train=True, download=True, transform=pipeline )
test_set = datasets.MNIST("data",train=False, download=True, transform=pipeline )

train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True)

C:\Users\17513\AppData\Roaming\Python\Python39\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
#5、构建网络模型

"""
模型设计：开始->卷积->激活->池化->卷积->激活->池化->全连接->激活->全连接->结束

"""
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,10,5)
        self.conv2 = nn.Conv2d(10,20,3)
        self.fc1 = nn.Linear(20*5*5,1000)
        self.fc2 = nn.Linear(1000,200)
        self.fc3 = nn.Linear(200,10)
        
    def forward(self,x):
        input_size = x.size(0) #拿到开始定义的超参数batch_size=32
        x = self.conv1(x) #输入为batch_size*1*28*28   输出batch_size*10*24*24 (28-5+1=24)
        x = F.relu(x)
        x = F.max_pool2d(x,2,2) #输出为batch_size*10*12*12
        
        x = self.conv2(x) #输出为batch_size*20*10*10（12-3+1=10）
        x = F.relu(x)
        x = F.max_pool2d(x,2,2) #输出为batch_size*20*5*5
        
        x = x.view(input_size, -1)#拉平
        
        x = self.fc1(x)#输出为batch_size*500
        x = F.relu(x)
        
        x = self.fc2(x)#输出为batch_size*10  ，10个类别
        x = F.relu(x)
        x = self.fc3(x)
        output = F.log_softmax(x,dim=1) #计算分类后每个数字的概率值
        return output
        

In [6]:
#6、定义优化器
model = Net().to(DEVICE)#创建一个模型并部署到设备上

optimizer = optim.Adam(model.parameters())

In [7]:
#7、定义训练方法
def train_model(model,device,train_loader,optimizer,epoch):
    #模型训练
    model.train()
    for batch_index, (data , target) in enumerate(train_loader):
        #部署到设备上
        data,target = data.to(device),target.to(device)
        #梯度初始化
        optimizer.zero_grad()
        #训练后的结果
        output = model(data)
        #计算损失
        loss=F.cross_entropy(output,target)
        #反向传播
        loss.backward()
        #参数优化
        optimizer.step()
        if batch_index % 3000 == 0:
            print("Train Epoch : {} \t Loss : {:.6f}".format(epoch,loss.item()))
            

In [8]:
#8、定义测试方法
def test_model(model,device,test_loader):
    #模型验证
    model.eval()
    #正确率
    correct = 0.0
    #测试损失
    test_loss = 0.0
    with torch.no_grad():
        for data,target in test_loader:
            #部署到设备上
            data,target = data.to(device),target.to(device)
            #测试数据
            output = model(data)
            #计算损失
            test_loss += F.cross_entropy(output,target).item()
            #找到概率值最大的下标
            pred=output.argmax(dim=1)
            #累计正确的值
            correct += pred.eq(target.view_as(pred)).sum().item()
        test_loss /= len(test_loader.dataset)
        print("Test --- Average  loss : {:.4f},Accuracy : {:.3f}\n".format(test_loss,100.0*correct/len(test_loader.dataset)))
        


In [9]:
#9、调用7和8
for epoch in range(1,EPOCHS + 1):
    train_model(model,DEVICE,train_loader,optimizer,epoch)
    test_model(model,DEVICE,test_loader)

C:\Users\17513\AppData\Roaming\Python\Python39\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Train Epoch : 1 	 Loss : 2.308303
Test --- Average  loss : 0.0004,Accuracy : 98.310

Train Epoch : 2 	 Loss : 0.037628
Test --- Average  loss : 0.0004,Accuracy : 98.520

Train Epoch : 3 	 Loss : 0.048102
Test --- Average  loss : 0.0003,Accuracy : 98.830

Train Epoch : 4 	 Loss : 0.024838
Test --- Average  loss : 0.0002,Accuracy : 98.940



face parsing

thermal

谷歌学术

LETPUB